In [40]:
!pip install simpletransformers

In [41]:
import simpletransformers

ModuleNotFoundError: No module named 'simpletransformers'

In [57]:
import pandas as pd
import nltk
import re
import string
import os
from tqdm import tqdm
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import IterableDataset, DataLoader, Dataset
import torch.optim as optim

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score   
import gc

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AutoTokenizer, DistilBertForSequenceClassification

In [58]:
train_data = pd.read_csv("EI-reg-En-anger-train.txt", delimiter="\t")
test_data = pd.read_csv("2018-EI-reg-En-anger-test-gold.txt", delimiter="\t")

In [59]:
train_data

,ID,Tweet,Affect Dimension,Intensity Score
0,2017-En-10264,@xandraaa5 @amayaallyn6 shut up hashtags are c...,anger,0.562
1,2017-En-10072,it makes me so fucking irate jesus. nobody is ...,anger,0.750
2,2017-En-11383,Lol Adam the Bull with his fake outrage...,anger,0.417
3,2017-En-11102,@THATSSHAWTYLO passed away early this morning ...,anger,0.354
4,2017-En-11506,@Kristiann1125 lol wow i was gonna say really?...,anger,0.438
...,...,...,...,...
1696,2017-En-11110,Got a $20 tip from a drunk Uber passenger. Tod...,anger,0.708
1697,2017-En-11497,@Claymakerbigsi @toghar11 @scott_mulligan_ @Bo...,anger,0.625
1698,2017-En-10539,@vladfucker69 i look rabid,anger,0.472
1699,2017-En-10468,"@m_t_f_72 I'm not surprised, I would be fuming! 😤",anger,0.479


### Preprocessing
Taken from the previous assignment, hence mostly uncommented.

In [60]:
def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return nltk.word_tokenize(text)

In [61]:
class Preprocessing():

    def __init__(self, full_dataset):
        self.full_dataset = full_dataset
        self.word_to_ix = {}
        self.ix_to_word = {}
        self.context_dataset = []
        self.vocab_size = None
        
    def convert_lowercase(self, x):
        x = x.lower()
        return x
        
    def remove_emoji(self, x):
        emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r'', x)
        
    exclude = string.punctuation
    
    def remove_punc(self, x):
        exclude = string.punctuation
        return x.translate(str.maketrans('', '', exclude))
    
    def remove_special_chars(self, x):
        x = re.sub('[^A-Za-z0-9]+', ' ', x)
        return x

    def remove_one_letter_words(self, x):
        x = re.sub(r'(?:^| )\w(?:$| )', ' ', x).strip()
        return x
    
    # default is to apply all these preprocessing steps
    def apply_preprocessing(self,
                            lowercase=True,
                            remove_emoji=True,
                            remove_punc=True,
                            remove_special_chars=True,
                            remove_one_letter_words=True):
        if lowercase:
            self.full_dataset['Tweet'] = self.full_dataset['Tweet'].apply(self.convert_lowercase)
        if remove_emoji:
            self.full_dataset['Tweet'] = self.full_dataset['Tweet'].apply(self.remove_emoji)
        if remove_punc:
            self.full_dataset['Tweet'] = self.full_dataset['Tweet'].apply(self.remove_punc)
        if remove_special_chars:
            self.full_dataset['Tweet'] = self.full_dataset['Tweet'].apply(self.remove_special_chars)
        if remove_one_letter_words:
            self.full_dataset['Tweet'] = self.full_dataset['Tweet'].apply(self.remove_one_letter_words)

In [62]:
cl1 = Preprocessing(train_data)
cl1.apply_preprocessing()
train_data_preprocessed = cl1.full_dataset

cl2 = Preprocessing(test_data)
cl2.apply_preprocessing()
test_data_preprocessed = cl2.full_dataset

In [66]:
train_data_preprocessed = train_data_preprocessed.drop(['ID', 'Affect Dimension'], axis=1)
test_data_preprocessed = test_data_preprocessed.drop(['ID', 'Affect Dimension'], axis=1)

In [67]:
train_data_preprocessed

,Tweet,Intensity Score
0,xandraaa5 amayaallyn6 shut up hashtags are coo...,0.562
1,it makes me so fucking irate jesus nobody is c...,0.750
2,lol adam the bull with his fake outrage,0.417
3,thatsshawtylo passed away early this morning i...,0.354
4,kristiann1125 lol wow was gonna say really hah...,0.438
...,...,...
1696,got 20 tip from drunk uber passenger today get...,0.708
1697,claymakerbigsi toghar11 scottmulligan boxingfa...,0.625
1698,vladfucker69 look rabid,0.472
1699,mtf72 im not surprised would be fuming,0.479


In [44]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english", 
                                                           num_labels=1,
                                                           ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

C:\Users\Damja\anaconda3\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Damja\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [72]:
tokenized_data_train = train_data_preprocessed['Tweet'].apply(tokenizer)
tokenized_data_test = test_data_preprocessed['Tweet'].apply(tokenizer)


In [76]:
tokenized_data_train

0       [input_ids, attention_mask]
1       [input_ids, attention_mask]
2       [input_ids, attention_mask]
3       [input_ids, attention_mask]
4       [input_ids, attention_mask]
                   ...             
1696    [input_ids, attention_mask]
1697    [input_ids, attention_mask]
1698    [input_ids, attention_mask]
1699    [input_ids, attention_mask]
1700    [input_ids, attention_mask]
Name: Tweet, Length: 1701, dtype: object

In [80]:
for idx, item in enumerate(tokenized_data_train):
    item['labels'] = train_data_preprocessed['Intensity Score'][idx]

In [ ]:
for idx, item in enumerate(tokenized_data_test):
    item['labels'] = test_data_preprocessed['Intensity Score'][idx]

Create a list of dicts, where each entry is a dictionary. In each dictionary we have the input_ids, attention_mask, and labels in form of torch tensors.

In [107]:
data_train_full = []
for item in tokenized_data_train:
    itemlist={}
    for key in item.keys():
        if key == 'labels':
            itemlist[key] = torch.tensor(item[key], dtype=torch.double)
        else:
            itemlist[key] = torch.tensor(item[key], dtype=torch.long)
    data_train_full.append(itemlist)

In [ ]:
data_test_full = []
for item in tokenized_data_test:
    itemlist={}
    for key in item.keys():
        if key == 'labels':
            itemlist[key] = torch.tensor(item[key], dtype=torch.double)
        else:
            itemlist[key] = torch.tensor(item[key], dtype=torch.long)
    data_test_full.append(itemlist)

In [108]:
data_train_full[0]

{'input_ids': tensor([  101,  1060, 29159, 11057,  2629, 25933,  3148,  3973,  2078,  2575,
          3844,  2039, 23325, 15900,  2015,  2024,  4658, 15807,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': tensor(0.5620, dtype=torch.float64)}

In [111]:
batch_size=32
args = TrainingArguments(
    #evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    #per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    report_to="none",
    weight_decay=0.01,
    #metric_for_best_model='accuracy',
    output_dir='results',
    logging_dir='logs',
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=data_train_full,
    tokenizer=tokenizer,
)

PyTorch: setting up devices


In [112]:
trainer.train()

***** Running training *****
  Num examples = 1701
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 162
  Number of trainable parameters = 66954241


Step,Training Loss


KeyboardInterrupt: 

Get the the length of the largest tweet, so we can create a padding for all tweets that contain fewer words. Also, let us define the needed funtion to create the padding. This is done as pytorch needs the inputs to be of the same length.